## Training

In [5]:
import pandas as pd
import re
import jieba
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.sparse import hstack, csr_matrix
import math

file_name = 'train.csv' 
file_path = file_name 

df = pd.read_csv(file_path)

print(f"數據集形狀 (列數, 行數): {df.shape}")
print("數據集前 5 行:")
print(df.head())

數據集形狀 (列數, 行數): (495387, 2)
數據集前 5 行:
          name  price
0   ! 十八麻油鴨粽-熱     50
1  ! 大福 小米肉粽-凍    104
2    ! 筍芋竹香粽-凍    298
3   ! 總 蛋黃肉粽-熱     42
4   ! 鮑魚干貝荷飯-凍    258


In [ ]:
# ----------------------------------------
# 0.1 從商品名稱提取初始價格特徵
# ----------------------------------------
price_pattern = re.compile(r'(?i)(?:\$|nt\$?)\s*(\d+(?:\.\d+)?)')

def extract_initial_price(name):
    """從商品名稱提取以 $ 或 NT 開頭的價格數值。若未找到價格，回傳 0。"""
    if pd.isna(name):
        return 0.0
    match = price_pattern.search(str(name))
    if match:
        try:
            return float(match.group(1))
        except ValueError:
            return 0.0
    return 0.0

df['initial_price'] = df['name'].apply(extract_initial_price).fillna(0.0)
print("--- 初始價格特徵提取結果 ---")
print(df[['name', 'initial_price']].head())

# ----------------------------------------
# 0.2 綜合文本清洗函數 (保留貨幣符號)
# ----------------------------------------
def comprehensive_clean_name_v3(name):
    """保留貨幣符號、數字、字母、中文並移除其他符號。"""
    name = str(name).strip().lower()
    name = name.replace("&quot;", " ")
    name = re.sub(r'[^$\u4e00-\u9fa5a-z0-9\s]', ' ', name)
    name = re.sub(r"\s+", " ", name).strip()
    return name

df['cleaned_name'] = df['name'].apply(comprehensive_clean_name_v3)
print("\n--- 綜合文本清洗結果 (保留貨幣符號) ---")
print(df[['name', 'cleaned_name']].head())

# ----------------------------------------
# 0.3 目標變量轉換 (Price -> Log Price)
# ----------------------------------------
df['log_price'] = np.log1p(df['price'])
print(f"\n目標變量 Log 轉換完成，新欄位為 'log_price'")


--- 初始價格特徵提取結果 ---
          name  initial_price
0   ! 十八麻油鴨粽-熱            0.0
1  ! 大福 小米肉粽-凍            0.0
2    ! 筍芋竹香粽-凍            0.0
3   ! 總 蛋黃肉粽-熱            0.0
4   ! 鮑魚干貝荷飯-凍            0.0

--- 綜合文本清洗結果 (保留貨幣符號) ---
          name cleaned_name
0   ! 十八麻油鴨粽-熱     十八麻油鴨粽 熱
1  ! 大福 小米肉粽-凍    大福 小米肉粽 凍
2    ! 筍芋竹香粽-凍      筍芋竹香粽 凍
3   ! 總 蛋黃肉粽-熱     總 蛋黃肉粽 熱
4   ! 鮑魚干貝荷飯-凍     鮑魚干貝荷飯 凍

目標變量 Log 轉換完成，新欄位為 'log_price'


In [7]:
# ----------------------------------------
# 1. 中文分詞 (Tokenization)
# ----------------------------------------
# 建議您安裝 jieba: !pip install jieba

stopwords = set()  # 可在此載入領域停用詞

def segment_chinese_text(text):
    """使用 Jieba 進行精確模式分詞，並移除停用詞。"""
    words = jieba.cut(text, cut_all=False)
    tokens = [word for word in words if word.strip() != "" and word not in stopwords]
    return " ".join(tokens)

df['name_tokens'] = df['cleaned_name'].apply(segment_chinese_text)
print("--- 中文分詞結果 ---")
print(df[['cleaned_name', 'name_tokens']].head())

# ----------------------------------------
# 2. 準備訓練數據 (同時分割文本與初始價格特徵)
# ----------------------------------------
X_train_text, X_test_text, X_train_initial_price, X_test_initial_price, y_train, y_test = train_test_split(
    df['name_tokens'],
    df['initial_price'],
    df['log_price'],
    test_size=0.2,
    random_state=42
)

print(f"數據集分割完成: 訓練集大小={X_train_text.shape[0]}, 測試集大小={X_test_text.shape[0]}")


Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/pk/knrkn_dn3l38qxkbzxpx06g40000gn/T/jieba.cache
Loading model cost 0.372 seconds.
Prefix dict has been built successfully.


--- 中文分詞結果 ---
  cleaned_name  name_tokens
0     十八麻油鴨粽 熱  十八 麻油 鴨 粽 熱
1    大福 小米肉粽 凍  大福 小米 肉 粽 凍
2      筍芋竹香粽 凍    筍芋 竹香 粽 凍
3     總 蛋黃肉粽 熱    總 蛋黃肉 粽 熱
4     鮑魚干貝荷飯 凍   鮑魚 干貝 荷飯 凍
數據集分割完成: 訓練集大小=396309, 測試集大小=99078


In [8]:
# ----------------------------------------
# 3. TF-IDF 向量化
# ----------------------------------------

# 按指示使用指定參數進行向量化
tfidf = TfidfVectorizer(
    max_features=50000,
    ngram_range=(1, 2)
)

# 在訓練集上 fit (學習詞彙表和權重)
X_train_vec = tfidf.fit_transform(X_train_text)

# 在測試集上 transform (使用訓練集的詞彙表)
X_test_vec = tfidf.transform(X_test_text)

print(f"\n--- TF-IDF 向量化結果 ---")
print(f"總詞彙表大小 (特徵維度): {X_train_vec.shape[1]}")
print(f"訓練集特徵矩陣形狀: {X_train_vec.shape}")
print(f"測試集特徵矩陣形狀: {X_test_vec.shape}")

# ----------------------------------------
# 3.1 數值特徵標準化與特徵拼接
# ----------------------------------------
scaler = StandardScaler()
X_train_price_scaled = scaler.fit_transform(X_train_initial_price.to_numpy().reshape(-1, 1))
X_test_price_scaled = scaler.transform(X_test_initial_price.to_numpy().reshape(-1, 1))

# 將標準化後的數值特徵轉為稀疏矩陣並與 TF-IDF 特徵拼接
X_train_final = hstack([X_train_vec, csr_matrix(X_train_price_scaled)])
X_test_final = hstack([X_test_vec, csr_matrix(X_test_price_scaled)])

print(f"--- 特徵拼接結果 ---")
print(f"訓練集最終特徵矩陣形狀: {X_train_final.shape}")
print(f"測試集最終特徵矩陣形狀: {X_test_final.shape}")

# ----------------------------------------
# 4. 模型訓練 (Ridge Regression)
# ----------------------------------------
ridge_model = Ridge(alpha=1.0)

print("--- 模型訓練 (Ridge) ---")
# 使用結合後的特徵進行訓練
ridge_model.fit(X_train_final, y_train)
print("Ridge 模型訓練完成。")

# 保留模型實例供後續使用
model = ridge_model



--- TF-IDF 向量化結果 ---
總詞彙表大小 (特徵維度): 50000
訓練集特徵矩陣形狀: (396309, 50000)
測試集特徵矩陣形狀: (99078, 50000)
--- 特徵拼接結果 ---
訓練集最終特徵矩陣形狀: (396309, 50001)
測試集最終特徵矩陣形狀: (99078, 50001)
--- 模型訓練 (Ridge) ---
Ridge 模型訓練完成。


In [9]:
# ----------------------------------------
# 5. 模型評估
# ----------------------------------------

# 使用最終特徵矩陣進行預測
y_pred_log = model.predict(X_test_final)

# 轉換回原始價格 (Price)
y_pred = np.expm1(y_pred_log)
y_true = np.expm1(y_test)

# 計算評估指標
mae = mean_absolute_error(y_true, y_pred)
rmse = math.sqrt(mean_squared_error(y_true, y_pred))

print(f"\n--- 模型評估 ---")
print(f"平均絕對誤差 (MAE) - 原始價格: {mae:.2f}")
print(f"均方根誤差 (RMSE) - 原始價格: {rmse:.2f}")

# 示範預測結果
results_df = pd.DataFrame({
    'Actual_Price': y_true,
    'Predicted_Price': y_pred
})
print("部分預測結果:")
print(results_df.head())



--- 模型評估 ---
平均絕對誤差 (MAE) - 原始價格: 709.30
均方根誤差 (RMSE) - 原始價格: 5342.11
部分預測結果:
        Actual_Price  Predicted_Price
412125         230.0       235.760484
207881       41310.0     34267.467955
353876        1199.0       737.647625
210982         890.0       620.221040
300354         495.0       807.706882


## Testing

In [10]:
# 匯入必要的庫
import pandas as pd
import re
import jieba
import numpy as np
import os
from scipy.sparse import csr_matrix, hstack

# ----------------------------------------------------
# 1. 讀取測試集
# ----------------------------------------------------
test_file_path = 'test.csv'
try:
    df_test = pd.read_csv(test_file_path)
    print(f"成功讀取測試集: {test_file_path}")
    print(f"測試集形狀: {df_test.shape}")
except FileNotFoundError:
    print(f"錯誤: 找不到檔案 '{test_file_path}'。正在創建演示數據。")
    df_test = pd.DataFrame({
        'name': [
            '! 亞洲特級香米-熱',
            '"奇妮"最新款醫療用束帶(未滅菌) GH20',
            'NT$1590品牌女運動鞋-2025版',
            '$99 玩具積木'
        ]
    })


# ----------------------------------------
# 2. 定義清洗與特徵工程函數 (需與訓練階段一致)
# ----------------------------------------
price_pattern = re.compile(r'(?i)(?:\$|nt\$?)\s*(\d+(?:\.\d+)?)')

def extract_initial_price(name):
    if pd.isna(name):
        return 0.0
    match = price_pattern.search(str(name))
    if match:
        try:
            return float(match.group(1))
        except ValueError:
            return 0.0
    return 0.0

def comprehensive_clean_name_v3(name):
    name = str(name).strip().lower()
    name = name.replace("&quot;", " ")
    name = re.sub(r'[^$\u4e00-\u9fa5a-z0-9\s]', ' ', name)
    name = re.sub(r"\s+", " ", name).strip()
    return name

stopwords = set()

def segment_chinese_text(text):
    words = jieba.cut(text, cut_all=False)
    tokens = [word for word in words if word.strip() != "" and word not in stopwords]
    return " ".join(tokens)


成功讀取測試集: test.csv
測試集形狀: (55043, 1)


In [11]:
# ----------------------------------------
# 1. 在測試集上套用清洗、分詞與初始價格提取
# ----------------------------------------
df_test['initial_price'] = df_test['name'].apply(extract_initial_price).fillna(0.0)
df_test['cleaned_name'] = df_test['name'].apply(comprehensive_clean_name_v3)
df_test['name_tokens'] = df_test['cleaned_name'].apply(segment_chinese_text)

print("\n--- 測試集清洗與特徵工程結果 (前 5 筆) ---")
print(df_test[['name', 'initial_price', 'name_tokens']].head())


# ----------------------------------------
# 2. 應用 TF-IDF 轉換與數值特徵標準化
# ----------------------------------------
if 'tfidf' not in locals():
    raise NameError("錯誤: 找不到 'tfidf' 變數。請先運行訓練集的程式碼。")
if 'scaler' not in locals():
    raise NameError("錯誤: 找不到 'scaler' 變數。請先運行訓練集的程式碼。")

X_test_vec_final = tfidf.transform(df_test['name_tokens'])
initial_price_scaled = scaler.transform(df_test["initial_price"].to_numpy().reshape(-1, 1))
X_test_final = hstack([X_test_vec_final, csr_matrix(initial_price_scaled)])

print(f"\n--- 測試集特徵矩陣 ---")
print(f"TF-IDF 特徵矩陣形狀: {X_test_vec_final.shape}")
print(f"最終特徵矩陣形狀: {X_test_final.shape}")


# ----------------------------------------
# 3. 模型預測 (使用已訓練好的 Ridge 模型)
# ----------------------------------------
if 'model' not in locals():
    raise NameError("錯誤: 找不到 'model' 變數。請先運行訓練集的程式碼。")

# 預測 log_price 並還原至原始價格
y_pred_log_test = model.predict(X_test_final)
y_pred_price_test = np.expm1(y_pred_log_test)
df_test['price'] = y_pred_price_test



--- 測試集清洗與特徵工程結果 (前 5 筆) ---
                          name  initial_price                   name_tokens
0                    !(呷)蛋黃粽-熱            0.0                      呷 蛋黃 粽 熱
1  #13-PILOT可擦印章 FRIXION stamp            0.0  13 pilot 可擦 印章 frixion stamp
2  #17-PILOT可擦印章 FRIXION stamp            0.0  17 pilot 可擦 印章 frixion stamp
3                #304隔熱湯碗組16CM            0.0              304 隔熱 湯碗 組 16cm
4               #316方形筷盒組-20cm            0.0              316 方形 筷盒 組 20cm

--- 測試集特徵矩陣 ---
TF-IDF 特徵矩陣形狀: (55043, 50000)
最終特徵矩陣形狀: (55043, 50001)


In [12]:
# ----------------------------------------
# 4. 輸出結果
# ----------------------------------------

print("\n--- 最終預測結果 (前 5 筆) ---")
print(df_test[['name', 'price']].head())

# 儲存結果到 CSV 文件
output_filename = 'test_predictions.csv'
df_test[['name', 'price']].to_csv(output_filename, index=False, float_format='%.2f')

print(f"\n預測結果已儲存到 '{output_filename}'")


--- 最終預測結果 (前 5 筆) ---
                          name       price
0                    !(呷)蛋黃粽-熱  174.916932
1  #13-PILOT可擦印章 FRIXION stamp  102.425269
2  #17-PILOT可擦印章 FRIXION stamp   91.428596
3                #304隔熱湯碗組16CM  166.532301
4               #316方形筷盒組-20cm  173.735604

預測結果已儲存到 'test_predictions.csv'


## version 2

In [17]:
import pandas as pd
import re
import jieba
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np

# ------------------------------------------------
# 🔧 停用詞
english_stopwords = set(stopwords.words("english"))
chinese_stopwords = {
    "的", "了", "和", "是", "我", "你", "他", "她", "它",
    "我們", "你們", "他們", "它們", "這個", "那個", "全新", "現貨", "免運",
    "商品", "原裝", "保證", "超值", "最新", "促銷", "限量", "特價", "正品", "包郵"
}
stopwords_all = english_stopwords | chinese_stopwords

# ------------------------------------------------
# 🧹 預處理函式
def preprocess_product_name(text: str) -> str:
    if not isinstance(text, str):
        text = str(text)
    text = text.lower()
    text = re.sub(r"([a-z0-9]+)", r" \1 ", text)
    text = re.sub(r"[^a-z0-9\u4e00-\u9fff ]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    words = jieba.lcut(text)
    words = [w for w in words if w.strip() and w not in stopwords_all]
    return " ".join(words)

# ------------------------------------------------
# 🧠 建立 Pipeline
pipeline = Pipeline([
    ("tfidf", TfidfVectorizer(
        preprocessor=preprocess_product_name,
        max_features=5000,
        ngram_range=(1, 2)
    )),
    ("model", RidgeCV(alphas=np.logspace(-2, 2, 10), cv=5))
])

# ------------------------------------------------
# 🧮 示範資料
df = pd.read_csv("train.csv")
# ------------------------------------------------
# 🚀 訓練模型
pipeline.fit(df["name"], df["price"])

# ------------------------------------------------
# 📊 預測與評估
y_pred = pipeline.predict(df["name"])
print("R²:", r2_score(df["price"], y_pred))
print("MAE:", mean_absolute_error(df["price"], y_pred))
print("RMSE:", mean_squared_error(df["price"], y_pred))

# ------------------------------------------------
# 🔍 觀察 TF-IDF 權重最高的詞
tfidf = pipeline.named_steps["tfidf"]
model = pipeline.named_steps["model"]

feature_names = np.array(tfidf.get_feature_names_out())
coef_df = pd.DataFrame({
    "feature": feature_names,
    "coef": model.coef_
}).sort_values(by="coef", ascending=False)

print("\n💰 提高價格的關鍵詞：")
print(coef_df.head(10))

print("\n💸 降低價格的關鍵詞：")
print(coef_df.tail(10))

R²: 0.023200342530669937
MAE: 1295.4936923276314
RMSE: 711398059.5035032

💰 提高價格的關鍵詞：
            feature          coef
1758             健行  69415.961909
1363  timberland 男款  68057.706298
1362     timberland  52275.260413
2517             小麥  40187.663504
843              i7  19090.434224
1583             中筒  17847.292231
3754             磨砂  17430.304845
330            512g  16654.348128
3617             男款  15609.939726
4352             變頻  15564.050020

💸 降低價格的關鍵詞：
      feature         coef
689     edwin -2488.680280
1239  sandisk -2505.327958
3424       濾網 -2508.083641
2195    固態 硬碟 -2519.377740
2037    原廠 公司 -2920.643262
3721       短袖 -2935.592689
3663       皮套 -3029.479875
1724       保護 -3198.117542
1732      保護殼 -3479.190625
3636       登山 -5041.630944


In [18]:
test_df = pd.read_csv("test.csv")
y_test_pred = pipeline.predict(test_df["name"])
test_df["price"] = y_test_pred
print(test_df.head())
test_df.to_csv("test_predictions.csv", index=False)

                          name        price
0                    !(呷)蛋黃粽-熱  1223.280674
1  #13-PILOT可擦印章 FRIXION stamp  2593.867919
2  #17-PILOT可擦印章 FRIXION stamp  3068.384807
3                #304隔熱湯碗組16CM   442.106147
4               #316方形筷盒組-20cm   670.524270


In [ ]:
import pandas as pd
import numpy as np
import re
import jieba
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import make_scorer # 用於自訂評分器
import math

# =============================================
# 🔧 1️⃣ 自訂前處理 Transformer
# =============================================

price_pattern = re.compile(r'(?i)(?:\$|nt\$?)\s*(\d+(?:\.\d+)?)')

class PriceExtractor(BaseEstimator, TransformerMixin):
    """從商品名稱提取初始價格"""
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        def extract_initial_price(name):
            if pd.isna(name):
                return 0.0
            match = price_pattern.search(str(name))
            if match:
                try:
                    return float(match.group(1))
                except ValueError:
                    return 0.0
            return 0.0
        prices = X.apply(extract_initial_price).fillna(0.0).to_numpy().reshape(-1, 1)
        return prices


class NameCleaner(BaseEstimator, TransformerMixin):
    """清洗與分詞商品名稱"""
    def __init__(self, stopwords=None):
        # ❗ 不要用可變的 set() 當預設值
        # 改成用 None 並在 fit() 初始化
        self.stopwords = stopwords

    def fit(self, X, y=None):
        if self.stopwords is None:
            self.stopwords_ = frozenset()  # 不可變集合
        else:
            self.stopwords_ = frozenset(self.stopwords)
        return self

    def transform(self, X):
        cleaned = []
        for name in X:
            name = str(name).strip().lower()
            name = name.replace("&quot;", " ")
            name = re.sub(r'[^$\u4e00-\u9fa5a-z0-9\s]', ' ', name)
            name = re.sub(r"\s+", " ", name).strip()
            words = jieba.cut(name, cut_all=False)
            tokens = [w for w in words if w.strip() and w not in self.stopwords_]
            cleaned.append(" ".join(tokens))
        return cleaned


# =============================================
# ⚙️ 2️⃣ Pipeline 定義
# =============================================

# 文字特徵管線：清洗＋TF-IDF
text_pipeline = Pipeline([
    ("cleaner", NameCleaner()),
    ("tfidf", TfidfVectorizer(max_features=50000, ngram_range=(1, 2)))
])

# 數值特徵管線：提取初始價格＋標準化
price_pipeline = Pipeline([
    ("extract", PriceExtractor()),
    ("scale", StandardScaler(with_mean=False))
])

# 組合特徵
preprocessor = ColumnTransformer([
    ("text", text_pipeline, "name"),
    ("price", price_pipeline, "name")
])

# 主 Pipeline
model_pipeline = Pipeline([
    ("preprocess", preprocessor),
    ("model", Ridge(alpha=1.0))
])

# =============================================
# 🧪 3️⃣ 資料載入與訓練
# =============================================

df = pd.read_csv("train.csv")

# log 價格作為目標
df["log_price"] = np.log1p(df["price"])

X_train, X_test, y_train, y_test = train_test_split(df, df["log_price"], test_size=0.2, random_state=42)

# 訓練
model_pipeline.fit(X_train, y_train)

# 預測
y_pred_log = model_pipeline.predict(X_test)
y_pred = np.expm1(y_pred_log)
y_true = np.expm1(y_test)

# =============================================
# 📊 4️⃣ 模型評估
# =============================================
mae = mean_absolute_error(y_true, y_pred)
rmse = math.sqrt(mean_squared_error(y_true, y_pred))

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")

results = pd.DataFrame({
    "Actual": y_true,
    "Predicted": y_pred
})
print(results.head())


MAE: 845.26
RMSE: 5608.26
         Actual     Predicted
412125    230.0    291.289140
207881  41310.0  33411.087323
353876   1199.0   1501.927664
210982    890.0    542.505490
300354    495.0    783.392727


In [24]:
import pandas as pd
import numpy as np
import re
import jieba
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error, make_scorer
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from scipy.sparse import csr_matrix
import math

# =============================================
# 🎯 0️⃣ 競賽評估標準：SMAPE 函數
# =============================================

def smape(y_true_log, y_pred_log):
    """
    Symmetric Mean Absolute Percentage Error (SMAPE)
    用於 GridSearchCV 評估和最終結果報告。
    """
    # 1. 將 log 價格還原為原始價格
    y_true_orig = np.expm1(y_true_log)
    y_pred_orig = np.expm1(y_pred_log)
    
    # 2. 避免預測負值
    y_pred_orig[y_pred_orig < 0] = 0
    
    # 3. 計算 SMAPE
    numerator = np.abs(y_pred_orig - y_true_orig)
    denominator = (np.abs(y_true_orig) + np.abs(y_pred_orig)) / 2
    
    # 避免除以零
    non_zero_mask = denominator != 0
    smape_val = np.mean(numerator[non_zero_mask] / denominator[non_zero_mask]) * 100
    
    return smape_val

# 將 SMAPE 轉換為 Sklearn 評分器（GridSearchCV 預設最大化分數，因此取負值）
smap_scorer = make_scorer(smape, greater_is_better=False)


# =============================================
# 🔧 1️⃣ 自訂前處理 Transformer
# =============================================

price_pattern = re.compile(r'(?i)(?:\$|nt\$?)\s*(\d+(?:\.\d+)?)')

class PriceExtractor(BaseEstimator, TransformerMixin):
    """從商品名稱提取初始價格"""
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        def extract_initial_price(name):
            if pd.isna(name):
                return 0.0
            match = price_pattern.search(str(name))
            if match:
                try:
                    return float(match.group(1))
                except ValueError:
                    return 0.0
            return 0.0
        # 輸出為 NumPy 陣列，符合 Scikit-learn 慣例
        prices = X.apply(extract_initial_price).fillna(0.0).to_numpy().reshape(-1, 1)
        return prices


class NameCleaner(BaseEstimator, TransformerMixin):
    """清洗與分詞商品名稱，並移除增強停用詞"""
    def __init__(self, stopwords=None):
        # 修正: 僅設置實例屬性，不做任何參數修改
        # 如果 stopwords 是可變的，則複製一份
        self.stopwords = list(stopwords) if stopwords is not None else None
        
        # 這是固定的預設停用詞，可以在 fit 中與 self.stopwords 結合
        self.default_stopwords = {
            '的', '了', '是', '在', '我', '有', '個', '組', '件', '入', 
            '組', '顆', '款', '型', '代', '號', '新', '舊', '版', '免運', 
            '特價', '促銷', '熱銷', '買', '送', '加購', '限定', '超值',
            '超商', '現貨', '預購', '全新', '特惠', '大號', '小號', '套裝'
        }

    def fit(self, X, y=None):
        # 修正: 在 fit 階段執行結合和轉換為 frozenset
        final_stopwords = self.default_stopwords.copy()
        if self.stopwords is not None:
            final_stopwords.update(self.stopwords)
            
        self.stopwords_ = frozenset(final_stopwords)
        return self

    def transform(self, X):
        cleaned = []
        for name in X:
            name = str(name).strip().lower()
            name = name.replace("&quot;", " ")
            # 保留貨幣符號、中文字、字母、數字和空格
            name = re.sub(r'[^$\u4e00-\u9fa5a-z0-9\s]', ' ', name)
            name = re.sub(r"\s+", " ", name).strip()
            
            # 使用 Jieba 分詞
            words = jieba.cut(name, cut_all=False)
            tokens = [w for w in words if w.strip() and w not in self.stopwords_]
            cleaned.append(" ".join(tokens))
        return cleaned


# =============================================
# ⚙️ 2️⃣ Pipeline 定義
# =============================================

# 文字特徵管線：清洗＋TF-IDF (包含 Bigram N-grams)
text_pipeline = Pipeline([
    ("cleaner", NameCleaner()),
    ("tfidf", TfidfVectorizer(max_features=50000, ngram_range=(1, 2)))
])

# 數值特徵管線：提取初始價格＋標準化
price_pipeline = Pipeline([
    ("extract", PriceExtractor()),
    ("scale", StandardScaler()) # 移除 with_mean=False，使用標準的 StandardScaler
])

# 組合特徵：ColumnTransformer 負責將不同欄位傳給不同的 Pipeline
preprocessor = ColumnTransformer([
    # 'name' 欄位同時用於文本處理和價格提取
    ("text", text_pipeline, "name"), 
    ("price", price_pipeline, "name")
])

# 主 Pipeline：前處理＋模型
model_pipeline = Pipeline([
    ("preprocess", preprocessor),
    # 這裡的 Ridge 模型只是一個佔位符，最終會被 GridSearchCV 調整
    ("model", Ridge(random_state=42)) 
])

# =============================================
# 🧪 3️⃣ 資料載入與訓練 (使用 GridSearchCV 進行調優)
# =============================================

# 假設 'train.csv' 在當前工作目錄
try:
    df = pd.read_csv("train.csv")
except FileNotFoundError:
    print("錯誤: 找不到 'train.csv'，請確認檔案路徑。")
    exit()

# log 價格作為目標
df["log_price"] = np.log1p(df["price"])

# 數據分割：使用 80% 訓練集，20% 測試集進行本地評估
X_train, X_test, y_train, y_test = train_test_split(df, df["log_price"], test_size=0.2, random_state=42)

# --- 參數調優設定 ---
param_grid = {
    # 'model' 是 Pipeline 中 Ridge 模型的名稱
    'model__alpha': [0.1, 1.0, 10.0, 50.0, 100.0, 500.0] 
}

# 定義 K-Fold 交叉驗證策略
cv_strategy = KFold(n_splits=5, shuffle=True, random_state=42)

# 創建 GridSearchCV，使用 SMAPE 作為評分標準
grid_search = GridSearchCV(
    estimator=model_pipeline, 
    param_grid=param_grid,
    scoring=smap_scorer, # 使用 SMAPE 評分器
    cv=cv_strategy,
    verbose=2,
    n_jobs=-1
)

print("--- 訓練與調優開始：Ridge + GridSearchCV (優化 SMAPE) ---")
grid_search.fit(X_train, y_train)

# 獲取最佳模型
best_model = grid_search.best_estimator_
best_alpha = best_model.named_steps['model'].alpha
best_cv_smape = -grid_search.best_score_

print("\n--- 訓練結果 ---")
print(f"最佳 Ridge alpha: {best_alpha}")
print(f"最佳 5-Fold CV SMAPE: {best_cv_smape:.4f}%")


# =============================================
# 📊 4️⃣ 模型評估與最終報告
# =============================================

# 使用最佳模型進行預測
y_pred_log = best_model.predict(X_test)

# 轉換回原始價格
y_pred = np.expm1(y_pred_log)
y_true = np.expm1(y_test)

# 計算最終評估指標
mae = mean_absolute_error(y_true, y_pred)
rmse = math.sqrt(mean_squared_error(y_true, y_pred))
final_smape = smape(y_test, y_pred_log) 

print("\n--- 測試集最終評估 ---")
print(f"Symmetric Mean Absolute Percentage Error (SMAPE): {final_smape:.4f}%")
print(f"平均絕對誤差 (MAE): {mae:.2f}")
print(f"均方根誤差 (RMSE): {rmse:.2f}")

results = pd.DataFrame({
    "Actual": y_true,
    "Predicted": y_pred
})
print("\n部分預測結果 (前 5 筆):")
print(results.head().round(2))

# =============================================
# 📤 5️⃣ 測試集預測與檔案輸出 (為 Kaggle 提交準備)
# =============================================

try:
    df_test = pd.read_csv("test.csv")
    print("\n--- 正在為測試集生成最終預測結果 ---")
    
    # 使用最佳模型對完整的測試集進行預測
    test_pred_log = best_model.predict(df_test)
    test_pred_price = np.expm1(test_pred_log)
    
    # 確保價格不為負值
    test_pred_price[test_pred_price < 0] = 0
    
    # 創建提交文件 (假設測試集包含 'item_id' 或其他唯一識別欄位)
    # 這裡假設您的測試集只需要 'name' 和預測的 'price'
    
    submission_df = pd.DataFrame({
        "name": df_test["name"],
        "price": test_pred_price.round(2)
    })
    
    output_filename = 'kaggle_submission_ridge_optimized.csv'
    submission_df.to_csv(output_filename, index=False)
    
    print(f"\n✅ 提交檔案已成功儲存至: '{output_filename}'")
    
except FileNotFoundError:
    print("\n⚠️ 找不到 'test.csv'，跳過生成最終提交檔案步驟。")


--- 訓練與調優開始：Ridge + GridSearchCV (優化 SMAPE) ---
Fitting 5 folds for each of 6 candidates, totalling 30 fits


Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/pk/knrkn_dn3l38qxkbzxpx06g40000gn/T/jieba.cache
Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/pk/knrkn_dn3l38qxkbzxpx06g40000gn/T/jieba.cache
Loading model from cache /var/folders/pk/knrkn_dn3l38qxkbzxpx06g40000gn/T/jieba.cache
Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/pk/knrkn_dn3l38qxkbzxpx06g40000gn/T/jieba.cache
Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/pk/knrkn_dn3l38qxkbzxpx06g40000gn/T/jieba.cache
Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/pk/knrkn_dn3l38qxkbzxpx06g40000gn/T/jieba.cache
Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/pk/knrkn_dn3l38qxkbzxpx06g40000gn/T/jieba.cache
Loading model cost 0.576 se

[CV] END ...................................model__alpha=1.0; total time=  40.3s
[CV] END ...................................model__alpha=1.0; total time=  40.0s
[CV] END ...................................model__alpha=1.0; total time=  40.6s
[CV] END ...................................model__alpha=0.1; total time=  41.0s
[CV] END ...................................model__alpha=0.1; total time=  41.3s
[CV] END ...................................model__alpha=0.1; total time=  41.9s
[CV] END ...................................model__alpha=0.1; total time=  42.0s
[CV] END ...................................model__alpha=0.1; total time=  42.4s
[CV] END ...................................model__alpha=1.0; total time=  39.3s
[CV] END ..................................model__alpha=10.0; total time=  39.6s
[CV] END ..................................model__alpha=10.0; total time=  39.9s
[CV] END ..................................model__alpha=10.0; total time=  39.8s
[CV] END ...................

/var/folders/pk/knrkn_dn3l38qxkbzxpx06g40000gn/T/ipykernel_72849/2175959613.py:27: RuntimeWarning: overflow encountered in expm1


[CV] END .................................model__alpha=500.0; total time=  31.8s
[CV] END .................................model__alpha=500.0; total time=  32.4s
[CV] END .................................model__alpha=500.0; total time=  32.2s
[CV] END .................................model__alpha=500.0; total time=  32.4s

--- 訓練結果 ---
最佳 Ridge alpha: 1.0
最佳 5-Fold CV SMAPE: 47.9974%

--- 測試集最終評估 ---
Symmetric Mean Absolute Percentage Error (SMAPE): 47.7161%
平均絕對誤差 (MAE): 708.82
均方根誤差 (RMSE): 5339.09

部分預測結果 (前 5 筆):
         Actual  Predicted
412125    230.0     238.84
207881  41310.0   35439.79
353876   1199.0     718.67
210982    890.0     608.49
300354    495.0     798.33

--- 正在為測試集生成最終預測結果 ---

✅ 提交檔案已成功儲存至: 'kaggle_submission_ridge_optimized.csv'
